### Подготовка

In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

from lets_plot import *
from lets_plot.mapping import as_discrete
LetsPlot.setup_html()

import os; import sys; sys.path.append(os.path.join(sys.path[0], "..", ".."))
from util import util

In [2]:
def plot_matrix(plots=[], width=400, height=300, columns=2):
    bunch = GGBunch()
    for i in range(len(plots)):
        row = int(i / columns)
        column = i % columns
        bunch.add_plot(plots[i], column * width, row * height, width, height)
    return bunch.show()

In [3]:
char_widths_df = util.get_df("../../data/char_widths.csv", "train")
char_widths_df

,char_id,char,alphabet,font_face,font_size,font_version,width
0,65,A,basic_latin,Courier,9,,9
1,66,B,basic_latin,Courier,9,,9
2,67,C,basic_latin,Courier,9,,9
3,68,D,basic_latin,Courier,9,,9
4,69,E,basic_latin,Courier,9,,9
...,...,...,...,...,...,...,...
97939,38754,面,japanese,Verdana,20,bi,43
97940,38761,革,japanese,Verdana,20,bi,43
97941,38936,領,japanese,Verdana,20,bi,43
97942,39080,風,japanese,Verdana,20,bi,43


In [4]:
char_widths_s = char_widths_df.groupby(["font_face", "font_size", "font_version", "char"]).width.median().astype(int)
char_widths_s

font_face  font_size  font_version  char
Courier    9                                 9
                                    !        9
                                    "        9
                                    #        9
                                    $        9
                                            ..
Verdana    20         i             面       42
                                    革       42
                                    領       42
                                    風       42
                                    高       42
Name: width, Length: 90384, dtype: int64

## Шаг №1

Ширины символов пробразуются в "порядки": для каждого символа `X` каждого алфавита и для каждого шрифта `F` порядок - это количество символов этого же алфавита, которые по ширине меньше чем он сам, обозначаемое через `p(X, F)`. После для каждого символа вычисляется `p(X)` - среднее значение `p(X, F)` по всем `F` для данного `X`.

In [5]:
char_orders_df = util.transform_char_widths_to_orders(char_widths_df)
char_orders_df

alphabet                         basic_latin                               \
char                                            !     "     #     $     %   
font_face font_size font_version                                            
Courier   9                              NaN  NaN   NaN   NaN   NaN   NaN   
                    b                    NaN  NaN   NaN   NaN   NaN   NaN   
                    bi                   NaN  NaN   NaN   NaN   NaN   NaN   
                    i                    NaN  NaN   NaN   NaN   NaN   NaN   
          11                             NaN  NaN   NaN   NaN   NaN   NaN   
...                                      ...  ...   ...   ...   ...   ...   
Verdana   17        i                    3.0  5.0  19.0  82.0  36.0  94.0   
          20                             3.0  8.0  12.0  82.0  38.0  94.0   
                    b                    1.0  6.0  21.0  81.0  37.0  94.0   
                    bi                   0.0  6.0  19.0  76.0  30.0  94.0   
                    i                    3.0  5.0  15.0  82.0  31.0  93.0   

alphabet                                                 ... russian        \
char                                 &    '     (     )  ...       ц     ч   
font_face font_size font_version                         ...                 
Courier   9                        NaN  NaN   NaN   NaN  ...     NaN   NaN   
                    b              NaN  NaN   NaN   NaN  ...     NaN   NaN   
                    bi             NaN  NaN   NaN   NaN  ...     NaN   NaN   
                    i              NaN  NaN   NaN   NaN  ...     NaN   NaN   
          11                       NaN  NaN   NaN   NaN  ...     NaN   NaN   
...                                ...  ...   ...   ...  ...     ...   ...   
Verdana   17        i             65.0  2.0  14.0  14.0  ...    30.0  25.0   
          20                      74.0  0.0  12.0  12.0  ...    60.0  24.0   
                    b             81.0  0.0  14.0  14.0  ...    56.0  23.0   
                    bi            76.0  0.0  13.0  13.0  ...    41.0  21.0   
                    i             65.0  2.0  15.0  15.0  ...    43.0  22.0   

alphabet                                                                     \
char                                 ш      щ     ъ     ы     ь     э     ю   
font_face font_size font_version                                              
Courier   9                        NaN    NaN   NaN   NaN   NaN   NaN   NaN   
                    b              NaN    NaN   NaN   NaN   NaN   NaN   NaN   
                    bi             NaN    NaN   NaN   NaN   NaN   NaN   NaN   
                    i              NaN    NaN   NaN   NaN   NaN   NaN   NaN   
          11                       NaN    NaN   NaN   NaN   NaN   NaN   NaN   
...                                ...    ...   ...   ...   ...   ...   ...   
Verdana   17        i             97.0   97.0  30.0  75.0  16.0  20.0  93.0   
          20                      98.0   99.0  60.0  85.0  22.0  20.0  95.0   
                    b             97.0  100.0  64.0  92.0  21.0  15.0  97.0   
                    bi            97.0   99.0  54.0  89.0  19.0  15.0  97.0   
                    i             96.0   99.0  43.0  81.0  18.0  18.0  93.0   

alphabet                                
char                                 я  
font_face font_size font_version        
Courier   9                        NaN  
                    b              NaN  
                    bi             NaN  
                    i              NaN  
          11                       NaN  
...                                ...  
Verdana   17        i             25.0  
          20                      24.0  
                    b             23.0  
                    bi            41.0  
                    i             43.0  

[168 rows x 583 columns]

In [6]:
mean_order_df = char_orders_df.describe().loc["mean"].groupby("char").mean().to_frame("mean_order").reset_index()
mean_order_df

,char,mean_order
0,,1.016204
1,!,7.072917
2,"""",20.559028
3,#,45.486111
4,$,34.513889
...,...,...
533,面,99.000000
534,革,99.000000
535,領,99.000000
536,風,99.000000


## Шаг №2

Полученные значения `p(X)` складываем ровно в `carts_count` корзин.

In [7]:
carts_count = 10

In [8]:
mean_order_df["cart_id"] = KMeans(n_clusters=carts_count, random_state=42).fit(mean_order_df[["mean_order"]]).labels_
cart_id_replaces = {
    k: i
    for i, k in enumerate(mean_order_df.groupby("cart_id").mean_order.median().sort_values().keys())
}
mean_order_df = mean_order_df.replace({"cart_id": cart_id_replaces})
mean_order_df

,char,mean_order,cart_id
0,,1.016204,0
1,!,7.072917,0
2,"""",20.559028,2
3,#,45.486111,5
4,$,34.513889,3
...,...,...,...
533,面,99.000000,9
534,革,99.000000,9
535,領,99.000000,9
536,風,99.000000,9


Рисуем `geom_boxplot()` с вариацией `p(X)` для каждой корзины:

In [9]:
ggplot(mean_order_df, aes(as_discrete("cart_id", order_by="..middle.."), "mean_order")) + geom_boxplot()

Рисуем `geom_bar()` с `p(X)` для каждого `X` каждой корзины:

In [10]:
plot_matrix([
    ggplot(mean_order_df[mean_order_df.cart_id == cart_id]) + \
        geom_bar(aes(as_discrete("char", order_by="mean_order", order=1), "mean_order"), stat='identity') + \
        coord_flip() + ylim(0, 105) + \
        ggtitle("cart_id = {0}".format(cart_id))
    for cart_id in range(carts_count - 1, -1, -1)
])

## Шаг №3

Фиксируем следующие значения:

- `basic_cart_id` - индекс средней по ширине корзины.

- `basic_font` - "Lucida Grande", 20pt, без модификаций.

- `cart_widths` - словарь, в котором каждая корзина получает в качестве ширины медианное значение ширин (в пикселях) всех символов корзины, начертанных базовым шрифтом.

- `basic_width` - значение из `cart_widths`, принадлежащее корзине с индексом `basic_cart_id`.

In [11]:
basic_cart_id = int(carts_count / 2)
print("Basic cart_id:", basic_cart_id)
basic_font = ("Lucida Grande", 20, "")
print("Basic font:", basic_font)
cart_widths = {
    cart_id: int(char_widths_s.loc[basic_font].loc[mean_order_df[mean_order_df.cart_id == cart_id].char.values].median())
    for cart_id in range(carts_count)
}
print("Cart widths:", cart_widths)
basic_width = cart_widths[basic_cart_id]
print("Basic width:", basic_width)

Basic cart_id: 5
Basic font: ('Lucida Grande', 20, '')
Cart widths: {0: 12, 1: 20, 2: 22, 3: 24, 4: 22, 5: 27, 6: 32, 7: 27, 8: 30, 9: 39}
Basic width: 27


## Шаг №4

Вычисляются нормировочные коэффициенты различных шрифтов. Для каждого `F` его нормировочный коэффициент `N(F)` - это отношение медианной ширины "базовой корзины" начертанной шрифтом `F` к `basic_width`.

In [12]:
ncoeff_s = char_widths_s.to_frame().reset_index().set_index("char").loc[mean_order_df[mean_order_df.cart_id == basic_cart_id].char.values].reset_index().groupby(["font_face", "font_size", "font_version"]).width.median() / basic_width
ncoeff_s

font_face  font_size  font_version
Courier    9                          0.333333
                      b               0.370370
                      bi              0.370370
                      i               0.333333
           11                         0.444444
                                        ...   
Verdana    17         i               1.000000
           20                         1.148148
                      b               1.259259
                      bi              1.259259
                      i               1.185185
Name: width, Length: 168, dtype: float64

## Шаг №5

С помощью нормировочных коэффициентов восстанавливаются прогнозируемые ширины символов в пикселях: для каждого символа `X` и шрифта `F` шириной становится ширина корзины которой принадлежит символ, умноженная на нормировочный коэффициент данного шрифта.

In [13]:
char_widths_calc_df = char_widths_s.to_frame().reset_index()\
    .merge(mean_order_df, on="char")\
    .merge(ncoeff_s.to_frame().reset_index(), \
           on=["font_face", "font_size", "font_version"], \
           suffixes=("_original", "_ncoeff"))\
    .assign(width_calc=lambda r: (np.round(r.cart_id.map(cart_widths) * r.width_ncoeff)).astype(int))
char_widths_calc_df.head()

,font_face,font_size,font_version,char,width_original,mean_order,cart_id,width_ncoeff,width_calc
0,Courier,9,,,9,1.016204,0,0.333333,4
1,Courier,9,,!,9,7.072917,0,0.333333,4
2,Courier,9,,"""",9,20.559028,2,0.333333,7
3,Courier,9,,#,9,45.486111,5,0.333333,9
4,Courier,9,,$,9,34.513889,3,0.333333,8


In [14]:
mean_width_df = char_widths_calc_df.groupby(["cart_id", "char"])[["width_original", "width_calc"]].mean()\
    .assign(width_diff=lambda r: r.width_calc - r.width_original).reset_index()
mean_width_df

,cart_id,char,width_original,width_calc,width_diff
0,0,,8.392857,8.571429,0.178571
1,0,!,9.779762,8.571429,-1.208333
2,0,',8.095238,8.571429,0.476190
3,0,",",8.845238,8.571429,-0.273810
4,0,.,8.678571,8.571429,-0.107143
...,...,...,...,...,...
533,9,面,26.154762,27.839286,1.684524
534,9,革,26.154762,27.839286,1.684524
535,9,領,26.154762,27.839286,1.684524
536,9,風,26.154762,27.839286,1.684524


График ошибок: для каждого символа `X` каждой корзины рисуется разница между спрогнозированной шириной и фактической.

In [15]:
plot_matrix([
    ggplot(mean_width_df[mean_width_df.cart_id == cart_id], \
           aes(as_discrete("char", order_by="width_diff", order=1), "width_diff")) + \
    geom_bar(stat="identity", sampling=sampling_pick(538)) + \
    coord_flip() + ylim(-8, 6) + \
    ggtitle("cart_id = {0}".format(cart_id))
    for cart_id in range(carts_count - 1, -1, -1)
])

## Шаг №6

Второй `geom_boxplot()` с вариацией спрогнозированных ширин символов для каждой корзины.

In [16]:
ggplot(char_widths_calc_df, aes(as_discrete("cart_id", order=-1), "width_calc")) + geom_boxplot()